In [10]:
# Necessary imports

from pathlib import Path

from torch.utils.data import Dataset  # not the one from PyG!
from torch_geometric.loader import DataLoader
import torch

from torch_geometric.nn import GCNConv
from torch.nn.functional import relu, binary_cross_entropy_with_logits, linear


In [16]:
# MyDataset class for handling file loading

class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()
        self.graphs = list(path.glob("*.pt"))
    
    def __getitem__(self, idx):
        return torch.load(self.graphs[idx])
    
    def __len__(self) -> int:
        return len(self.graphs)

In [46]:
# Load first batch (batch_1_0)

dataset = MyDataset(Path("./dataset/batch_1_0"))

# Preliminary analysis of dataset behavior

print("Number of graphs in set:", len(dataset))
print("Set of keys in each graph", dataset[0].keys)
print("Number of nodes in first graph:", dataset[0].num_nodes)
print("Number of edges in first graph:", dataset[0].num_edges)
print("Number of node features:", dataset[0].num_node_features)
print("Number of edge features:", dataset[0].num_edge_features)
print("Is this graph undirected? ", dataset[0].is_undirected())

print("Number of edge features:", dataset[0]['y'])

loader = DataLoader(dataset, batch_size=32)

Number of graphs in set: 1000
Set of keys in each graph ['y', 'edge_attr', 'x', 'edge_index']
Number of nodes in first graph: 419
Number of edges in first graph: 4882
Number of node features: 6
Number of edge features: 4
Is this graph undirected?  False
Number of edge features: tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [69]:
class GNN(torch.nn.Module):
    def __init__(self, sample, hidden_layers = [8]):
        super().__init__()

        self.conv = []

        if len(hidden_layers) == 0:
            # Case when no hidden layers are passed
            self.conv.append(GCNConv(sample.num_node_features, 1))

        else:
            # Case when hidden layers are passed
            self.conv.append(GCNConv(sample.num_node_features, hidden_layers[0]))
            for i in range(len(hidden_layers) - 1):
                self.conv.append(GCNConv(hidden_layers[i], hidden_layers[i + 1]))    
            self.conv.append(GCNConv(hidden_layers[-1], 1))
        
        self.conv_module = torch.nn.ModuleList(self.conv)

    def forward(self, data):

        x, edge_attr, edge_index = data.x, data.edge_attr, data.edge_index
        '''
        # Concatenate the edge attributes with the source node attributes
        src_node_attrs = x[edge_index[0]]
        conv_attributes = torch.cat((edge_attr, src_node_attrs), dim=1)    
        '''
    
        for i in range(len(self.conv)):
            print(i)
            print(len(x))
            print(len(edge_index[0]))
            print(len(edge_attr))
            x = self.conv[i](x, edge_index, edge_attr)
            if i < len(self.conv) - 1:
                x = relu(x)
        
        print("Wow")
        edge_inputs = torch.cat([x[edge_index[0]], x[edge_index[1]], edge_attr], dim=1)
        edge_scores = linear(edge_inputs, out_channels=1).squeeze()

        return edge_scores

In [70]:
device = torch.device('cpu')
model = GNN(dataset[0]).to(device)
data = dataset[0].to(device)
print(list(model.parameters()))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = binary_cross_entropy_with_logits(out, data.y)
    loss.backward()
    optimizer.step()



[Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True), Parameter containing:
tensor([[ 0.3662,  0.6250, -0.1496, -0.2941,  0.3087,  0.1807],
        [ 0.4392, -0.0926, -0.4628, -0.6398,  0.2015,  0.2732],
        [-0.5728,  0.0389, -0.0252, -0.4467, -0.4094, -0.3001],
        [ 0.4804, -0.0504, -0.4206, -0.2922,  0.2959, -0.6482],
        [ 0.6135, -0.1331,  0.1454, -0.0187, -0.4434, -0.3983],
        [ 0.0069, -0.1619, -0.1289, -0.6223,  0.5372, -0.5946],
        [ 0.1501,  0.0037, -0.4213, -0.2938,  0.0634, -0.5762],
        [-0.2775,  0.2389,  0.2953, -0.5232,  0.3022,  0.4491]],
       requires_grad=True), Parameter containing:
tensor([0.], requires_grad=True), Parameter containing:
tensor([[-0.1917, -0.4653, -0.4165, -0.0752, -0.3142, -0.3794, -0.5647, -0.6712]],
       requires_grad=True)]
0
tensor([[  0.0778,   0.7999,  -1.4975,  -3.6513, -10.3963,   7.5592],
        [  0.0810,   0.8275,  -1.5025,  -3.6144, -10.5768,   6.3697],
        [  0.0776,   

RuntimeError: The size of tensor a (21204) must match the size of tensor b (5301) at non-singleton dimension 0